![The Deep Purple Network Notebook App](https://raw.githubusercontent.com/greggtdd/DeepPurpleNetwork/master/app_images/dpnetwork_graph_banner.png)

In [1]:
######################################################################################################
# Project: The Deep Purple Network.                                                                  #
# (CC) 2020 Made by Gregorio Tedde just for analysis purpose.                                        #
#                                                                                                    #
#                                                                                                    #
#                                       PLEASE, DO NOT EDIT!                                         #
#                                                                                                    #
#                                                                                                    #
# Work License: Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License   #
# Libraries licenses in environment.yml                                                              #
# Git Repo: https://github.com/greggtdd/DeepPurpleNetwork                                            #
# Docs for widgets handling: https://bit.ly/2W2JPGm                                                  #
# Info: greggtedde@gmail.com                                                                         #
######################################################################################################

In [2]:
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from graph_bouncer import DataFramer, Networker
from IPython.display import display, clear_output

In [3]:
# Data frame handling
dp = DataFramer('https://raw.githubusercontent.com/greggtdd/DeepPurpleNetwork/master/dp_union_edges.csv')
dp.upload_df()
dp.density_sources()
dp.artist_research()

In [4]:
# Init state for the dropdown menu
artist = "Select an artist"

In [5]:
def unique_artist_source(array):
    """
    Returns the unique values for
    a specific source written as
    a query in the search box.
    
    Paramters
    ---------
    array: pandas.core.series.Series
        Desired column from the data frame.
    
    Returns
    -------
    list
    """
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, artist)
    return unique

dropdown_source = widgets.Dropdown(options = unique_artist_source(dp.data_['Source']))

In [6]:
# Widget - search output display
output_source = widgets.Output()

def dropdown_handler(change):
    output_source.clear_output()
    with output_source:
        output_text.value = dropdown_source.value
        display(dp.data_[dp.data_.Source == change.new])
        
dropdown_source.observe(dropdown_handler, names='value')

In [7]:
# Widget - search box
output_text = widgets.Text(value='',
                           placeholder='(e.g. Jon Lord)',
                           description="Artist:",
                           disabled=False)

def callback(widget):
    if output_text.value in dp.sour_:
        dropdown_source.value = output_text.value
        return widget.value
    
output_text.on_submit(callback)

In [8]:
# Request for error message image
response = requests.get('https://raw.githubusercontent.com/greggtdd/DeepPurpleNetwork/master/app_images/ritchie_message.png')
error_img = Image.open(BytesIO(response.content))

In [9]:
# Widget - rendered network graph
output_graph = widgets.Output()

def create_graph(_):
    if output_text.value in dp.sour_:
        dp.sub_framer(output_text.value)
        edges = dp.get_edge_data(sources=dp.sources_,
                                 targets=dp.targets_,
                                 weights=dp.weights_)
        nw = Networker(data=edges)
        nw.init_graph()
        nw.add_elements()
        nw.get_neighbors()
        nw.get_info()
        display(nw.show_graph())
    else:
        display(error_img)

In [10]:
# Widget - 'Create the graph' button
graph_button = widgets.Button(description='Create the graph')
output_start = widgets.Output()

def on_button_clicked(_):
    with output_start:
        with output_graph:
            output_start.clear_output()
            create_graph(_)
            
graph_button.on_click(on_button_clicked)

In [11]:
# Layout for the input widgets
item_layout = widgets.Layout(margin='20px 0 30px 150px')

In [12]:
# Tab for the output widgets
tab = widgets.Tab([output_graph, output_source])
tab.set_title(0, "Network Graph")
tab.set_title(1, "Subset Exploration")

<center>Digit an <b>Artist</b>'s name and press <code>Enter</code>on your keyboard.<br>
    If the name shows on the right, click on <code>Create the graph</code>.<br>
You can also select directly a name from the dropdown menu on the right and click <code>Create the graph</code>.<br>
<br>After loading is completed, scroll for zooming inside the network till you see the artists' names <br>and click on the nodes to see who their neighbors are. Drag and drop for moving across the graph.<br>
    The <i>Subset Exploration</i> tab shows each artist's universe used for building its neighborhood.<br><br>
    <a href="files/dp_last_graph.html">Click here</a> to switch the graph in full screen mode after creating it.</center>

In [13]:
input_widgets = widgets.HBox([output_text,
                              dropdown_source,
                              graph_button],
                             layout=item_layout)
display(input_widgets)
display(tab)

<center><a href="https://greggtdd.github.io/DeepPurpleNetwork/pages/graph">Return to the website</a></center>